![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/deployment/accelerated-models/accelerated-models-quickstart.png)

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Azure ML Hardware Accelerated Models Quickstart

This tutorial will show you how to deploy an image recognition service based on the ResNet 50 classifier using the Azure Machine Learning Accelerated Models service.  Get more information about our service from our [documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-accelerate-with-fpgas), [API reference](https://docs.microsoft.com/en-us/python/api/azureml-accel-models/azureml.accel?view=azure-ml-py), or [forum](https://aka.ms/aml-forum).

We will use an accelerated ResNet50 featurizer running on an FPGA. Our Accelerated Models Service handles translating deep neural networks (DNN) into an FPGA program.

For more information about using other models besides Resnet50, see the [README](./README.md).

The steps covered in this notebook are: 
1. [Set up environment](#set-up-environment)
* [Construct model](#construct-model)
    * Image Preprocessing
    * Featurizer (Resnet50)
    * Classifier
    * Save Model
* [Register Model](#register-model)
* [Convert into Accelerated Model](#convert-model)
* [Create Image](#create-image)
* [Deploy](#deploy-image)
* [Test service](#test-service)
* [Clean-up](#clean-up)

<a id="set-up-environment"></a>
## 1. Set up environment

In [ ]:
import os
import tensorflow as tf

### Retrieve Workspace
If you haven't created a Workspace, please follow [this notebook](https://github.com/Azure/MachineLearningNotebooks/blob/master/configuration.ipynb) to do so. If you have, run the codeblock below to retrieve it. 

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

<a id="construct-model"></a>
## 2. Construct model

There are three parts to the model we are deploying: pre-processing, featurizer with ResNet50, and classifier with ImageNet dataset. Then we will save this complete Tensorflow model graph locally before registering it to your Azure ML Workspace.

### 2.a. Image preprocessing
We'd like our service to accept JPEG images as input. However the input to ResNet50 is a tensor. So we need code that decodes JPEG images and does the preprocessing required by ResNet50. The Accelerated AI service can execute TensorFlow graphs as part of the service and we'll use that ability to do the image preprocessing. This code defines a TensorFlow graph that preprocesses an array of JPEG images (as strings) and produces a tensor that is ready to be featurized by ResNet50.

**Note:** Expect to see TF deprecation warnings until we port our SDK over to use Tensorflow 2.0.

In [ ]:
# Input images as a two-dimensional tensor containing an arbitrary number of images represented a strings
import azureml.accel.models.utils as utils
tf.reset_default_graph()

in_images = tf.placeholder(tf.string)
image_tensors = utils.preprocess_array(in_images)
print(image_tensors.shape)

### 2.b. Featurizer
We use ResNet50 as a featurizer. In this step we initialize the model. This downloads a TensorFlow checkpoint of the quantized ResNet50.

In [ ]:
from azureml.accel.models import QuantizedResnet50
save_path = os.path.expanduser('~/models')
model_graph = QuantizedResnet50(save_path, is_frozen = True)
feature_tensor = model_graph.import_graph_def(image_tensors)
print(model_graph.version)
print(feature_tensor.name)
print(feature_tensor.shape)

### 2.c. Classifier
The model we downloaded includes a classifier which takes the output of the ResNet50 and identifies an image. This classifier is trained on the ImageNet dataset. We are going to use this classifier for our service. The next [notebook](./accelerated-models-training.ipynb) shows how to train a classifier for a different data set. The input to the classifier is a tensor matching the output of our ResNet50 featurizer.

In [ ]:
classifier_output = model_graph.get_default_classifier(feature_tensor)
print(classifier_output)

### 2.d. Save Model
Now that we loaded all three parts of the tensorflow graph (preprocessor, resnet50 featurizer, and the classifier), we can save the graph and associated variables to a directory which we can register as an Azure ML Model.

In [ ]:
# model_name must be lowercase
model_name = "resnet50"
model_save_path = os.path.join(save_path, model_name)
print("Saving model in {}".format(model_save_path))

with tf.Session() as sess:
    model_graph.restore_weights(sess)
    tf.saved_model.simple_save(sess, model_save_path,
                                   inputs={'images': in_images},
                                   outputs={'output_alias': classifier_output})

### 2.e. Important! Save names of input and output tensors

These input and output tensors that were created during the preprocessing and classifier steps are also going to be used when **converting the model** to an Accelerated Model that can run on FPGA's and for **making an inferencing request**. It is very important to save this information! You can see our defaults for all the models in the [README](./README.md).

By default for Resnet50, these are the values you should see when running the cell below: 
* input_tensors = "Placeholder:0"
* output_tensors = "classifier/resnet_v1_50/predictions/Softmax:0"

In [ ]:
input_tensors = in_images.name
output_tensors = classifier_output.name

print(input_tensors)
print(output_tensors)

<a id="register-model"></a>
## 3. Register Model

You can add tags and descriptions to your models. Using tags, you can track useful information such as the name and version of the machine learning library used to train the model. Note that tags must be alphanumeric.

In [ ]:
from azureml.core.model import Model

registered_model = Model.register(workspace = ws,
                                  model_path = model_save_path,
                                  model_name = model_name)

print("Successfully registered: ", registered_model.name, registered_model.description, registered_model.version, sep = '\t')

<a id="convert-model"></a>
## 4. Convert Model

For conversion you need to provide names of input and output tensors. This information can be found from the model_graph you saved in step 2.e. above.

**Note**: Conversion may take a while and on average for FPGA model it is about 1-3 minutes and it depends on model type.

In [ ]:
from azureml.accel import AccelOnnxConverter

convert_request = AccelOnnxConverter.convert_tf_model(ws, registered_model, input_tensors, output_tensors)

if convert_request.wait_for_completion(show_output = False):
    # If the above call succeeded, get the converted model
    converted_model = convert_request.result
    print("\nSuccessfully converted: ", converted_model.name, converted_model.url, converted_model.version, 
          converted_model.id, converted_model.created_time, '\n')
else:
    print("Model conversion failed. Showing output.")
    convert_request.wait_for_completion(show_output = True)

<a id="create-image"></a>
## 5. Package the model into an Image

You can add tags and descriptions to image. Also, for FPGA model an image can only contain **single** model.

**Note**: The following command can take few minutes. 

In [ ]:
from azureml.core.image import Image
from azureml.accel import AccelContainerImage

image_config = AccelContainerImage.image_configuration()
# Image name must be lowercase
image_name = "{}-image".format(model_name)

image = Image.create(name = image_name,
                     models = [converted_model],
                     image_config = image_config, 
                     workspace = ws)
image.wait_for_creation(show_output = False)

<a id="deploy-image"></a>
## 6. Deploy
Once you have an Azure ML Accelerated Image in your Workspace, you can deploy it to two destinations, to a Databox Edge machine or to an AKS cluster. 

### 6.a. Databox Edge Machine using IoT Hub
See the sample [here](https://github.com/Azure-Samples/aml-real-time-ai/) for using the Azure IoT CLI extension for deploying your Docker image to your Databox Edge Machine.

### 6.b. Azure Kubernetes Service (AKS) using Azure ML Service
We are going to create an AKS cluster with FPGA-enabled machines, then deploy our service to it. For more information, see [AKS official docs](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where#aks).

#### Create AKS ComputeTarget

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget

# Uses the specific FPGA enabled VM (sku: Standard_PB6s)
# Standard_PB6s are available in: eastus, westus2, westeurope, southeastasia
prov_config = AksCompute.provisioning_configuration(vm_size = "Standard_PB6s",
                                                    agent_count = 1, 
                                                    location = "eastus")

aks_name = 'my-aks-pb6'
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

Provisioning an AKS cluster might take awhile (15 or so minutes), and we want to wait until it's successfully provisioned before we can deploy a service to it. If you interrupt this cell, provisioning of the cluster will continue. You can also check the status in your Workspace under Compute.

In [ ]:
%%time
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

#### Deploy AccelContainerImage to AKS ComputeTarget

In [ ]:
%%time
from azureml.core.webservice import Webservice, AksWebservice

# Set the web service configuration (for creating a test service, we don't want autoscale enabled)
# Authentication is enabled by default, but for testing we specify False
aks_config = AksWebservice.deploy_configuration(autoscale_enabled=False,
                                                num_replicas=1,
                                                auth_enabled = False)

aks_service_name ='my-aks-service'

aks_service = Webservice.deploy_from_image(workspace = ws,
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
aks_service.wait_for_deployment(show_output = True)

<a id="test-service"></a>
## 7. Test the service

### 7.a. Create Client
The image supports gRPC and the TensorFlow Serving "predict" API. We will create a PredictionClient from the Webservice object that can call into the docker image to get predictions. If you do not have the Webservice object, you can also create [PredictionClient](https://docs.microsoft.com/en-us/python/api/azureml-accel-models/azureml.accel.predictionclient?view=azure-ml-py) directly.

**Note:** If you chose to use auth_enabled=True when creating your AksWebservice, see documentation [here](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice(class)?view=azure-ml-py#get-keys--) on how to retrieve your keys and use either key as an argument to PredictionClient(...,access_token=key).
**WARNING:** If you are running on Azure Notebooks free compute, you will not be able to make outgoing calls to your service. Try locating your client on a different machine to consume it.

In [ ]:
# Using the grpc client in AzureML Accelerated Models SDK
from azureml.accel import client_from_service

# Initialize AzureML Accelerated Models client
client = client_from_service(aks_service)

You can adapt the client [code](https://github.com/Azure/aml-real-time-ai/blob/master/pythonlib/amlrealtimeai/client.py) to meet your needs. There is also an example C# [client](https://github.com/Azure/aml-real-time-ai/blob/master/sample-clients/csharp).

The service provides an API that is compatible with TensorFlow Serving. There are instructions to download a sample client [here](https://www.tensorflow.org/serving/setup).

### 7.b. Serve the model
To understand the results we need a mapping to the human readable imagenet classes

In [ ]:
import requests
classes_entries = requests.get("https://raw.githubusercontent.com/Lasagne/Recipes/master/examples/resnet50/imagenet_classes.txt").text.splitlines()

In [ ]:
# Score image with input and output tensor names
results = client.score_file(path="./snowleopardgaze.jpg", 
                             input_name=input_tensors, 
                             outputs=output_tensors)

# map results [class_id] => [confidence]
results = enumerate(results)
# sort results by confidence
sorted_results = sorted(results, key=lambda x: x[1], reverse=True)
# print top 5 results
for top in sorted_results[:5]:
    print(classes_entries[top[0]], 'confidence:', top[1])

<a id="clean-up"></a>
## 8. Clean-up
Run the cell below to delete your webservice, image, and model (must be done in that order). In the [next notebook](./accelerated-models-training.ipynb) you will learn how to train a classfier on a new dataset using transfer learning and finetune the weights.

In [ ]:
aks_service.delete()
aks_target.delete()
image.delete()
registered_model.delete()
converted_model.delete()